In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
import geopy.distance



In [2]:
conflict_df = pd.read_csv('../data/conflict_data_irq.csv')

In [3]:
conflict_df.drop(0)

,id,year,active_year,start_year,end_year,type_of_violence,conflict_new_id,conflict_name,dyad_new_id,dyad_name,...,date_prec,date_start,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,low,best,high
1,114064.0,2014,1.0,2014,2014,1.0,259.0,Iraq: Government,524.0,Government of Iraq - IS,...,2.0,2014-01-24,2014-01-25,0.0,0.0,1.0,7.0,8.0,8,8.0
2,62366.0,2013,1.0,2013,2013,3.0,506.0,IS - Civilians,973.0,IS - Civilians,...,1.0,2013-07-05,2013-07-05,0.0,0.0,0.0,7.0,7.0,7,7.0
3,114179.0,2014,1.0,2014,2014,1.0,259.0,Iraq: Government,524.0,Government of Iraq - IS,...,2.0,2014-02-21,2014-02-22,6.0,0.0,0.0,0.0,6.0,6,6.0
4,114432.0,2014,1.0,2014,2014,1.0,259.0,Iraq: Government,524.0,Government of Iraq - IS,...,2.0,2014-05-07,2014-05-08,0.0,0.0,0.0,7.0,7.0,7,7.0
5,143030.0,2014,1.0,2014,2014,1.0,259.0,Iraq: Government,524.0,Government of Iraq - IS,...,2.0,2014-09-30,2014-10-01,7.0,7.0,0.0,0.0,14.0,14,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7469,277061.0,2018,1.0,2018,2018,1.0,354.0,Turkey: Kurdistan,781.0,Government of Turkey - PKK,...,1.0,2018-12-14,2018-12-14,0.0,3.0,0.0,0.0,3.0,3,3.0
7470,277062.0,2018,1.0,2018,2018,1.0,354.0,Turkey: Kurdistan,781.0,Government of Turkey - PKK,...,1.0,2018-12-14,2018-12-14,0.0,2.0,0.0,0.0,2.0,2,2.0
7471,274527.0,2018,1.0,2018,2018,1.0,354.0,Turkey: Kurdistan,781.0,Government of Turkey - PKK,...,2.0,2018-12-14,2018-12-15,0.0,7.0,0.0,0.0,0.0,7,7.0
7472,274534.0,2018,1.0,2018,2018,1.0,259.0,Iraq: Government,524.0,Government of Iraq - IS,...,1.0,2018-12-22,2018-12-22,0.0,4.0,0.0,0.0,4.0,4,4.0


In [4]:
returnee_filepaths = [f for f in listdir("../data/returnees/") if f.endswith('.xlsx')]


In [ ]:

# master_filepaths = [s for s in listdir("../data/master_list/") if s.endswith('.xlsx')]
# mas_df = pd.concat((pd.read_excel("../data/master_list/"+s).assign(date=s[31:43]) 
                    for s in master_filepaths), ignore_index=True) 
#mas_df['date'] = pd.to_datetime(mas_df['date'])

In [5]:
ret_df = pd.concat((pd.read_excel("../data/returnees/"+f).assign(date=f[35:47])
                  for f in returnee_filepaths), ignore_index=True)

In [6]:
ret_df.dropna(how='all', inplace=True)

In [7]:
ret_df['date'] = pd.to_datetime(ret_df['date'])

In [8]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)



In [9]:
ret_df = trim_all_columns(ret_df)

In [10]:
ret_df.columns

Index(['Location ID', 'Place_ID', 'Governorate', 'District', 'Location_name',
       'Arabic_name', 'Latitude', 'Longitude', 'Returnee Families',
       'Returnee Individuals', 'Anbar', 'Babylon', 'Baghdad', 'Basrah',
       'Dahuk', 'Diyala', 'Erbil', 'Kerbala', 'Kirkuk', 'Missan', 'Muthanna',
       'Najaf', 'Ninewa', 'Qadissiya', 'Salahal Din', 'Sulaymaniyah',
       'Thi Qar', 'Wassit', 'Camp', 'Habitual Pre_31_October2018',
       'Habitual Residence (Habitable)', 'Habitual Residence (Uninhabitable)',
       'Host_families', 'Hotel_Motel', 'Informal_settlements', 'Other',
       'Religious_building', 'Rented_houses', 'School_building',
       'Unfinished_Abandoned_building', 'Unknown_shelter_type',
       'Pre June14 Period of displacement',
       'June July14 Period of displacement', 'August14 Period of displacement',
       'Post September 14 Period of displacement',
       'Post April15 Period of displacement',
       'Post March 16 Period of displacement',
       'Post 17 Oct

In [11]:

ret_df.sort_values(['Location_name', 'date'], inplace=True)




In [12]:
ret_df

,Location ID,Place_ID,Governorate,District,Location_name,Arabic_name,Latitude,Longitude,Returnee Families,Returnee Individuals,...,Pre June14 Period of displacement,June July14 Period of displacement,August14 Period of displacement,Post September 14 Period of displacement,Post April15 Period of displacement,Post March 16 Period of displacement,Post 17 October 16 Period of displacement,July 17 Period of displacement,Jan19,date
3639,2310031.0,33523.0,Baghdad,Tarmia,13 Al Dhabat-Abo Khameas,الضبات 13-قرية البو خميس,33.636245,44.19983,72.0,432.0,...,0.0,0.0,0.0,72.0,0.0,0.0,0.0,0.0,0.0,2018-10-31
43052,2310031.0,33523.0,Baghdad,Tarmia,13 Al Dhabat-Abo Khameas,الضبات 13-قرية البو خميس,33.636245,44.19983,72.0,432.0,...,0.0,0.0,0.0,72.0,0.0,0.0,0.0,0.0,0.0,2018-12-31
28348,2310031.0,33523.0,Baghdad,Tarmia,13 Al Dhabat-Abo Khameas,الضبات 13-قرية البو خميس,33.636245,44.19983,72.0,432.0,...,0.0,0.0,0.0,72.0,0.0,0.0,0.0,0.0,0.0,2019-02-28
79037,2310031.0,33523.0,Baghdad,Tarmia,13 Al Dhabat-Abo Khameas,الضبات 13-قرية البو خميس,33.636245,44.19983,74.0,444.0,...,0.0,0.0,0.0,74.0,0.0,0.0,0.0,0.0,0.0,2019-04-30
39196,2310031.0,33523.0,Baghdad,Tarmia,13 Al Dhabat-Abo Khameas,الضبات 13-قرية البو خميس,33.636245,44.19983,74.0,444.0,...,0.0,0.0,0.0,74.0,0.0,0.0,0.0,0.0,0.0,2019-06-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-30
87890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-31
75657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-31
64053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-12-31


In [13]:
ret_df.dropna(how='any', inplace=True)

In [14]:
ret_df['ret_delta'] = ret_df.groupby(['Location_name'])['Returnee Families'].transform(lambda x: x.diff()) 



In [15]:
ret_df.sort_index(inplace=True)

In [16]:
ret_df.sort_values(['Location_name', 'date'], inplace=True)

In [17]:
ret_df.reset_index(inplace=True)

In [18]:
ret_df

,index,Location ID,Place_ID,Governorate,District,Location_name,Arabic_name,Latitude,Longitude,Returnee Families,...,June July14 Period of displacement,August14 Period of displacement,Post September 14 Period of displacement,Post April15 Period of displacement,Post March 16 Period of displacement,Post 17 October 16 Period of displacement,July 17 Period of displacement,Jan19,date,ret_delta
0,28348,2310031.0,33523.0,Baghdad,Tarmia,13 Al Dhabat-Abo Khameas,الضبات 13-قرية البو خميس,33.636245,44.19983,72.0,...,0.0,0.0,72.0,0.0,0.0,0.0,0.0,0.0,2019-02-28,NaN
1,79037,2310031.0,33523.0,Baghdad,Tarmia,13 Al Dhabat-Abo Khameas,الضبات 13-قرية البو خميس,33.636245,44.19983,74.0,...,0.0,0.0,74.0,0.0,0.0,0.0,0.0,0.0,2019-04-30,2.0
2,39196,2310031.0,33523.0,Baghdad,Tarmia,13 Al Dhabat-Abo Khameas,الضبات 13-قرية البو خميس,33.636245,44.19983,74.0,...,0.0,0.0,74.0,0.0,0.0,0.0,0.0,0.0,2019-06-30,0.0
3,951,2310031.0,33523.0,Baghdad,Tarmia,13 Al Dhabat-Abo Khameas,الضبات 13-قرية البو خميس,33.636245,44.19983,74.0,...,0.0,0.0,74.0,0.0,0.0,0.0,0.0,0.0,2019-08-31,0.0
4,20584,2310031.0,33523.0,Baghdad,Tarmia,13 Al Dhabat-Abo Khameas,الضبات 13-قرية البو خميس,33.636245,44.19983,74.0,...,0.0,0.0,74.0,0.0,0.0,0.0,0.0,0.0,2019-10-31,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26237,56964,2107096.0,33953.0,Anbar,Heet,zawiat alwustaa,زوية الوسطى,33.543090,42.95544,705.0,...,301.0,0.0,200.0,204.0,0.0,0.0,0.0,0.0,2020-06-30,2.0
26238,86123,2107096.0,33953.0,Anbar,Heet,zawiat alwustaa,زوية الوسطى,33.543090,42.95544,705.0,...,301.0,0.0,200.0,204.0,0.0,0.0,0.0,0.0,2020-08-31,0.0
26239,73870,2107096.0,33953.0,Anbar,Heet,zawiat alwustaa,زوية الوسطى,33.543090,42.95544,705.0,...,301.0,0.0,200.0,204.0,0.0,0.0,0.0,0.0,2020-10-31,0.0
26240,62236,2107096.0,33953.0,Anbar,Heet,zawiat alwustaa,زوية الوسطى,33.543090,42.95544,705.0,...,301.0,0.0,200.0,204.0,0.0,0.0,0.0,0.0,2020-12-31,0.0


In [19]:
ret_df[ret_df['Location_name'] == 'Gwer']

,index,Location ID,Place_ID,Governorate,District,Location_name,Arabic_name,Latitude,Longitude,Returnee Families,...,June July14 Period of displacement,August14 Period of displacement,Post September 14 Period of displacement,Post April15 Period of displacement,Post March 16 Period of displacement,Post 17 October 16 Period of displacement,July 17 Period of displacement,Jan19,date,ret_delta
14022,35528,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1580.0,...,0.0,955.0,0.0,0.0,0.0,0.0,625.0,0.0,2018-10-15,NaN
14023,27988,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1824.0,...,0.0,1199.0,0.0,0.0,0.0,0.0,625.0,0.0,2019-02-28,244.0
14024,78671,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1824.0,...,0.0,1199.0,0.0,0.0,0.0,0.0,625.0,0.0,2019-04-30,0.0
14025,38829,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1824.0,...,0.0,1199.0,0.0,0.0,0.0,0.0,625.0,0.0,2019-06-30,0.0
14026,579,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1844.0,...,0.0,1219.0,0.0,0.0,0.0,0.0,625.0,0.0,2019-08-31,20.0
14027,20172,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1844.0,...,0.0,1219.0,0.0,0.0,0.0,0.0,625.0,0.0,2019-10-31,0.0
14028,29585,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,2226.0,...,0.0,1451.0,0.0,0.0,0.0,0.0,775.0,0.0,2019-12-31,382.0
14029,7400,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,2226.0,...,0.0,1451.0,0.0,0.0,0.0,0.0,775.0,0.0,2020-02-29,0.0
14030,10704,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,2226.0,...,0.0,1451.0,0.0,0.0,0.0,0.0,775.0,0.0,2020-04-29,0.0
14031,56667,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,2226.0,...,0.0,1451.0,0.0,0.0,0.0,0.0,775.0,0.0,2020-06-30,0.0


In [21]:
ret_df.at[14022, 'ret_delta']

nan

In [28]:
for i in range(len(ret_df)):
    if np.isnan(ret_df.at[i, 'ret_delta']):
        ret_df.at[i, 'ret_delta'] = ret_df.at[i, 'Returnee Families']
    else:
        None

In [29]:
ret_df[ret_df['Location_name'] == 'Gwer']

,index,Location ID,Place_ID,Governorate,District,Location_name,Arabic_name,Latitude,Longitude,Returnee Families,...,June July14 Period of displacement,August14 Period of displacement,Post September 14 Period of displacement,Post April15 Period of displacement,Post March 16 Period of displacement,Post 17 October 16 Period of displacement,July 17 Period of displacement,Jan19,date,ret_delta
14022,35528,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1580.0,...,0.0,955.0,0.0,0.0,0.0,0.0,625.0,0.0,2018-10-15,1580.0
14023,27988,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1824.0,...,0.0,1199.0,0.0,0.0,0.0,0.0,625.0,0.0,2019-02-28,244.0
14024,78671,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1824.0,...,0.0,1199.0,0.0,0.0,0.0,0.0,625.0,0.0,2019-04-30,0.0
14025,38829,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1824.0,...,0.0,1199.0,0.0,0.0,0.0,0.0,625.0,0.0,2019-06-30,0.0
14026,579,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1844.0,...,0.0,1219.0,0.0,0.0,0.0,0.0,625.0,0.0,2019-08-31,20.0
14027,20172,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1844.0,...,0.0,1219.0,0.0,0.0,0.0,0.0,625.0,0.0,2019-10-31,0.0
14028,29585,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,2226.0,...,0.0,1451.0,0.0,0.0,0.0,0.0,775.0,0.0,2019-12-31,382.0
14029,7400,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,2226.0,...,0.0,1451.0,0.0,0.0,0.0,0.0,775.0,0.0,2020-02-29,0.0
14030,10704,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,2226.0,...,0.0,1451.0,0.0,0.0,0.0,0.0,775.0,0.0,2020-04-29,0.0
14031,56667,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,2226.0,...,0.0,1451.0,0.0,0.0,0.0,0.0,775.0,0.0,2020-06-30,0.0


In [22]:
ret_df['ret_delta'] = ret_df.apply()

TypeError: 'float' object is not callable

In [ ]:
ret_df[ret_df['Location_name'] == 'Gwer']

In [ ]:
for i in ret_df.index.values:
    print (i, type(i))